In [2]:
import nibabel as nib #pip install nibabel
import h5py #pip install h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from scipy.spatial import ConvexHull
import json
from glob import glob
import os

import numpy as np
import nibabel as nib
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle

In [70]:
def get_center(streamlines):
    centers = np.zeros([len(streamlines),3])
    for i,sl in enumerate(streamlines):
        #print np.mean(sl,axis=0)
        centers[i,0] = np.mean(sl,axis=0)[0]
        centers[i,1] = np.mean(sl,axis=0)[1]
        centers[i,2] = np.mean(sl,axis=0)[2]
    the_center = np.mean(centers, axis=0)
    return the_center

def center_sls(streamlines):
    sls_centroid = get_center(streamlines)
    newsls =[]
    for i,sl in enumerate(streamlines):
        newsls.append(sl-sls_centroid)
    return newsls

def execute_qb(streamlines, dist_metric, smcl=5):
    qb = QuickBundles(threshold=dist_metric)
    clusters = qb.cluster(streamlines)
    print("Nb. clusters:", len(clusters))
    print("Cluster sizes:", map(len, clusters))
    print("N Small clusters:", sum(clusters < smcl))
    print("N Large clusters:", sum(clusters >= smcl))
    #print("Streamlines indices of the first cluster:\n", clusters[0].indices)
    #print("Centroid of the last cluster:\n", clusters[-1].centroid)
    #save_pickle('QB.pkl', clusters)
    return clusters

def get_qbparams(clusters, clszthr):
    return sum(clusters >= clszthr)

def execute_savecsv(path, clusters):
    import csv
    #clusters = clusters.get_large_clusters(50)
    clusters=clusters
    csvpath = path
    mycsv = csv.writer(open(csvpath,"w"), delimiter=',')
    mycsv.writerow(['dist_metric=', dist_metric, 'grand_tot=', len(clusters), 'tot=', len(clusters)])
    mycsv.writerow(['track','tot_cluster', 'clusternum', 'clustersz'])

    for i,cl in enumerate(clusters):
        mycsv.writerow([trkdir.split('/')[-1], len(clusters), i, len(cl)])

def savejson(clusters, savepath, bigclthr=5):
    clusters = clusters.get_large_clusters(bigclthr)
    # Select one of these:
    #streamline_idx = 20
    streamline = list(clusters[0])
    #json_streamline = json.dumps({streamline_idx:streamline.tolist()})
    jump_step = 5
    json_streamlines = {}
    json_bundles = {}
    # loop through all fibers 
    for bundle_idx in range(len(clusters)):
        bundle = list(clusters[bundle_idx])
        # loop through the selected bundle and put all lines into another variable
        for streamline_idx in range(0, len(bundle), jump_step):
            streamline = bundle[streamline_idx]
            streamline = streamline[1::5]
            json_streamlines[streamline_idx] = streamline.tolist()

        # then put this fiber into bundles set
        json_bundles[bundle_idx] = json_streamlines
        # and refresh json_streamlines
        json_streamlines = {}
    
    # print (len(json_bundles))
    # finally, write this bundles set into external file
    with open(savepath, 'w') as outfile:
        json.dump(json_bundles, outfile)

In [71]:
start_dist_metric = 50
trkdir = '/Users/kjordan/Desktop/SECOND_PAPER/CCI_demo/final_1/BA_Ex_Capsule_L.trk.trk'
trk,hdr = nib.trackvis.read(trkdir)
streamlines = [i[0] for i in trk]

In [75]:
centered_sls = center_sls(streamlines)
tier_0 = execute_qb(centered_sls, 31)
savejson(tier_1,'testing.json')

('Nb. clusters:', 10)
('Cluster sizes:', [2657, 1953, 1044, 89, 958, 48, 378, 580, 5, 291])
('N Small clusters:', 0)
('N Large clusters:', 10)


In [80]:
tier_1 = execute_qb(list(cluster_1[0]),17)
savejson(tier_2, 'data_partial.json')

('Nb. clusters:', 13)
('Cluster sizes:', [398, 657, 568, 23, 27, 184, 1, 13, 26, 18, 739, 1, 2])
('N Small clusters:', 3)
('N Large clusters:', 10)


In [84]:
def get_qbparams(clusters, clszthr):
    return sum(clusters >= clszthr)
def find_nbundles(streamlines, nbundles=10, initdistmetric=50):
    dmetric_list = [initdistmetric]
    distmetric = initdistmetric
    bundles = execute_qb(streamlines,50)
    Nbigcl = get_qbparams(bundles)
    while (Nbigcl != nbundles or Nbigcl != nbundles-1):
        if Nbigcl < nbundles:
            distmetric += int(distmetric/2)
            bundles = execute_qb(streamlines,50)
            Nbigcl = get_qbparams(bundles)
            dmetric_list.append(distmetric)
        else:
            distmetric -= int(distmetric/2)
            bundles = execute_qb(streamlines,50)
            Nbigcl = get_qbparams(bundles)
            dmetric_list.append(distmetric)
    return dmetric_list
    
    

In [85]:
find_nbundles(centered_sls)

('Nb. clusters:', 2)
('Cluster sizes:', [5748, 2255])
('N Small clusters:', 0)
('N Large clusters:', 2)


TypeError: get_qbparams() takes exactly 2 arguments (1 given)